In [1]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

In [2]:
# Load the dataset and apply data augmentation
data_dir = 'C:\Vijay\Capstone\Dataset'
img_height, img_width = 256, 256
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 481 images belonging to 5 classes.
Found 118 images belonging to 5 classes.


In [3]:
# Load ResNet50 base model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Create a custom CNN filter using the ResNet architecture
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)


In [4]:
# Combine the base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model to preserve the pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 25
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the trained model
model.save('fabric_classification_resnet_model.h5')

Epoch 1/25
15/15 [==============================] - 72s 5s/step - loss: 2.1218 - accuracy: 0.2205 - val_loss: 1.7128 - val_accuracy: 0.1562
Epoch 2/25
15/15 [==============================] - 70s 5s/step - loss: 1.7291 - accuracy: 0.3029 - val_loss: 1.6557 - val_accuracy: 0.2292
Epoch 3/25
15/15 [==============================] - 56s 4s/step - loss: 1.6016 - accuracy: 0.3000 - val_loss: 1.5480 - val_accuracy: 0.4375
Epoch 4/25
15/15 [==============================] - 53s 4s/step - loss: 1.5252 - accuracy: 0.3474 - val_loss: 1.5430 - val_accuracy: 0.1979
Epoch 5/25
15/15 [==============================] - 57s 4s/step - loss: 1.4915 - accuracy: 0.3562 - val_loss: 1.5053 - val_accuracy: 0.3229
Epoch 6/25
15/15 [==============================] - 55s 4s/step - loss: 1.4242 - accuracy: 0.4254 - val_loss: 1.5319 - val_accuracy: 0.3542
Epoch 7/25
15/15 [==============================] - 55s 4s/step - loss: 1.4008 - accuracy: 0.4232 - val_loss: 1.5582 - val_accuracy: 0.2917
Epoch 8/25
15/15 [==